In [1]:
import numpy as np
import pandas as pd

pd.set_option("display.max_colwidth", 1000)

In [ ]:
#import data_load
#reload(data_load)
#data_load.clean_text()

In [ ]:
products = data_load.get_clean_text()

In [ ]:
products['attributes_len'] = products['attributes'].map(lambda ss: len(ss))

In [ ]:
products['brand'] = products['attributes'].map(lambda kvs: kvs.get('mfg brand name', 'none'))
def b1g(s):
    a = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    if s == 'none':
        a[0] = 1.0
    elif s == 'unbranded':
        a[1] = 1.0
    elif s == 'hampton bay':
        a[2] = 1.0
    elif s == 'kohler':
        a[3] = 1.0
    elif s == 'everbilt':
        a[4] = 1.0
    elif s == 'home decorators collection':
        a[5] = 1.0
    elif s == 'ge':
        a[6] = 1.0
    return a

df_brand_features = products['brand'].apply(lambda s: pd.Series(b1g(s)))
df_brand_features.columns = ['brand_none', 'brand_unbranded', 'brand_hampton', 'brand_kohler', 'brand_ever', 'brand_home', 'brand_ge']
products = pd.concat([products, df_brand_features], axis=1)

In [ ]:
def f1g(q):
    if 'sxdli' in q:
        return 1.0
    else:
        return 0.0
products['corrected'] = products['queries'].map(lambda ss: { k:f1g(s) for k,s in ss.items() })

def f1(q):
    if 'sxdli' in q:
        return q[:-6]
    else:
        return q
products['queries'] = products['queries'].map(lambda ss: { k:f1(s) for k,s in ss.items() })

In [56]:
# length in words
products['queries_len'] = products['queries'].map(lambda kvs: {k:    max(1, len(v.split()))      for k,v in kvs.items()})
products['product_title_len'] = products['product_title'].map(lambda x: len(x.split()))
products['product_description_len'] = products['product_description'].map(lambda x:len(x.split()))
products['brand_len'] = products['brand'].map(lambda x:len(x.split()))
# length in words, that have no digits
no_number = re.compile(r"[0-9]")
products['queries_wlen'] = products['queries'].map(lambda kvs: {k:    max(1, len([x for x in v.split() if not no_number.search(x)]))      for k,v in kvs.items()})
products['product_title_wlen'] = products['product_title'].map(lambda x: len([x for x in x.split() if not no_number.search(x)]))
products['product_description_wlen'] = products['product_description'].map(lambda x:len([x for x in x.split() if not no_number.search(x)]))
products['brand_wlen'] = products['brand'].map(lambda x:len([x for x in x.split() if not no_number.search(x)]))
# length in letters
products['queries_let'] = products['queries'].map(lambda kvs: {k:    max(1, len(v))      for k,v in kvs.items()})
products['product_title_let'] = products['product_title'].map(lambda x: len(x))
products['product_description_let'] = products['product_description'].map(lambda x:len(x))
products['brand_let'] = products['brand'].map(lambda x:len(x))

In [58]:
import re
has_digit = re.compile(r"[0-9]")
def str_common_word(str1, str2):
    words, cnt, wcnt = str1.split(), 0, 0
    matched = set()
    wmatched = set()
    for word in words:
        if len(word) > 1 and str2.find(word)>=0:
            cnt+=1
            matched.add(word)
            if not has_digit.search(word):
                wcnt +=1
                wmatched.add(word)
    matched_len = sum([len(w) for w in matched])
    wmatched_len = sum([len(w) for w in wmatched])
    leftest = 15
    if wcnt > 0:
        other_words = str2.split()
        other_words.reverse()
        for i, w in enumerate(other_words):
            if words[-1] == w:
                leftest = i 
                break
    return [cnt, cnt*1.0/max(1.0, len(words)), matched_len, wcnt, wmatched_len, leftest]

def str_whole_word(str1, str2, i_):
    cnt = 0
    total_len = 0
    if len(str1) < 1:
        return [cnt, total_len]
    while i_ < len(str2):
        i_ = str2.find(str1, i_)
        if i_ == -1:
            return [cnt, total_len]
        else:
            cnt += 1
            i_ += len(str1)
            total_len += len(str1)
    return [cnt, total_len]

def bigram_match(str1, str2):
    words, intext, cnt = str1.split(), str2.split(), 0
    bi1 = zip(words, words[1:])
    bi2 = set(zip(intext, intext[1:]))
    matched = set()
    for x in bi1:
        if x in bi2:
            cnt+=1
            matched.add(x)
    matched_len = sum([len(w[0]) + len(w[1]) for w in matched])        
    return [cnt, matched_len]

def f_query_in(r):
    tit = r['product_title']
    desc = r['product_description']
    attrs = ' '.join([ k + ' ' + v for k,v in r['attributes'].items()])
    brand = r['brand']
    els = [tit, desc, attrs, brand]
    qs = r['queries']
    res = { k: [y    for el in els for x in [str_whole_word(v, el, 0) + str_common_word(v, el) + bigram_match(v, el)] for y in x] for k,v in qs.items()}
    return res

products['query_in_product_features'] = products.apply(f_query_in, axis=1)

In [ ]:
from collections import defaultdict

query_features = ['corrected', 'queries_len', 'queries_let', 'queries_wlen', 'query_in_product_features']

def combine_feats(r):
    feats = defaultdict(list)
    for c in query_features:
        ds = r[c]
        for k,v in ds.items():
            if type(v) == list:
                feats[k] += v
            else:
                feats[k] += [v]
    return feats

combined_query_features = products[query_features].apply(combine_feats, axis=1)

In [92]:
products['combined_query_features'] = combined_query_features
products.drop(query_features, axis=1, inplace=True)

In [64]:
direct_features = [
"brand_none",
"brand_unbranded",
"brand_hampton",
"brand_kohler",
"brand_ever",
"brand_home",
"brand_ge",
"attributes_len",
"product_title_len",
"product_description_len",
"brand_len",
"product_title_let",
"product_description_let",
"brand_let",
"product_title_wlen",
"product_description_wlen",
"brand_wlen"]

In [60]:
#pd.to_pickle(products, 'RAW_FEATURES')
#products = pd.read_pickle('RAW_FEATURES')

In [95]:
products.columns

Index([                 u'queries',            u'product_title',
            u'product_description',               u'attributes',
                          u'brand',               u'brand_none',
                u'brand_unbranded',            u'brand_hampton',
                   u'brand_kohler',               u'brand_ever',
                     u'brand_home',                 u'brand_ge',
                 u'attributes_len',        u'product_title_len',
        u'product_description_len',                u'brand_len',
              u'product_title_let',  u'product_description_let',
                      u'brand_let',       u'product_title_wlen',
       u'product_description_wlen',               u'brand_wlen',
        u'combined_query_features'],
      dtype='object')

In [114]:
product_columns = ['product_uid', 'product_title', 'product_description', 'brand']
def explode(r, rs):
    attrs = ' '.join([k + ' ' + v for k,v in r['attributes'].items()])
    qs = r['queries']
    for i,q in qs.items():
        new_row = [i, r['queries'][i]]
        new_row += [r[f] for f in product_columns]
        new_row += [attrs]
        new_row += [r[f] for f in direct_features]
        new_row += r['combined_query_features'][i]
        rs.append(new_row)
        
rows = []    
_ = products.reset_index().apply(lambda row: explode(row, rows), axis=1)
df_combined_query_features = pd.DataFrame(rows)
df_combined_query_features.columns = ['id', 'query'] + product_columns + ['attrs'] + direct_features + ['c%d' % i for i in range(44)]

In [118]:
queries = df_combined_query_features
queries.set_index('id', inplace=True)
pd.to_pickle(queries, 'FEATURES_WITH_TEXT_1')

In [122]:
queries

,query,product_uid,product_title,product_description,brand,attrs,brand_none,brand_unbranded,brand_hampton,brand_kohler,...,c34,c35,c36,c37,c38,c39,c40,c41,c42,c43
id,,,,,,,,,,,,,,,,,,,,,
1,90 unitsdegrees bracket,100001,simpson strong tie strongtie 12 gauge angle,only angles make joints stronger they also provide more consistent straight corners simpson strong tie strongtie offers wide variety angles various sizes thicknesses handle light duty lightduty jobs projects where structural connection needed bent skewed match project outdoor projects those where moisture present use our zmax zinc coate coated zinccoate zinccoated connectors provide extra resistance against corrosion look z end model number versatile connector various 90 connections home repair projects stronger than angled nailing screw fastening alone help ensure joints consistently straight strong dimensions 3 unitsinches xby 3 unitsinches xby 1 1d2 unitsinches made 12 gauge steel galvanized extra corrosion resistance install 10 d common nails 9 xby 1 1d2 unitsinches strong drive strongdrive sd screws,simpson strong tie strongtie,product width 3 bullet 02 stronger than angled nailing screw fastening alone bullet 01 versatile connector various 90 connections home repair projects bullet 06 galvanized extra corrosion resistance bullet 03 help ensure joints consistently straight strong material galvanized steel bullet 05 made 12 gauge steel number pieces 1 bullet 07 install 10 d common nails 9 xby 1 1d2 unitsinches strong drive strongdrive sd screws bullet 04 dimensions 3 unitsinches xby 3 unitsinches xby 1 1d2 unitsinches product height 3 gauge 12 mfg brand name simpson strong tie strongtie product depth 1p5 product weight lb 0p26,0.0,0.0,0.0,0.0,...,0,0,0,0.000000,0,0,0,15,0,0
2,angle bracket,100001,simpson strong tie strongtie 12 gauge angle,only angles make joints stronger they also provide more consistent straight corners simpson strong tie strongtie offers wide variety angles various sizes thicknesses handle light duty lightduty jobs projects where structural connection needed bent skewed match project outdoor projects those where moisture present use our zmax zinc coate coated zinccoate zinccoated connectors provide extra resistance against corrosion look z end model number versatile connector various 90 connections home repair projects stronger than angled nailing screw fastening alone help ensure joints consistently straight strong dimensions 3 unitsinches xby 3 unitsinches xby 1 1d2 unitsinches made 12 gauge steel galvanized extra corrosion resistance install 10 d common nails 9 xby 1 1d2 unitsinches strong drive strongdrive sd screws,simpson strong tie strongtie,product width 3 bullet 02 stronger than angled nailing screw fastening alone bullet 01 versatile connector various 90 connections home repair projects bullet 06 galvanized extra corrosion resistance bullet 03 help ensure joints consistently straight strong material galvanized steel bullet 05 made 12 gauge steel number pieces 1 bullet 07 install 10 d common nails 9 xby 1 1d2 unitsinches strong drive strongdrive sd screws bullet 04 dimensions 3 unitsinches xby 3 unitsinches xby 1 1d2 unitsinches product height 3 gauge 12 mfg brand name simpson strong tie strongtie product depth 1p5 product weight lb 0p26,0.0,0.0,0.0,0.0,...,0,0,0,0.000000,0,0,0,15,0,0
3,l bracket,100001,simpson strong tie strongtie 12 gauge angle,only angles make joints stronger they also provide more consistent straight corners simpson strong tie strongtie offers wide variety angles various sizes thicknesses handle light duty lightduty jobs projects where structural connection needed bent skewed match project outdoor projects those where moisture present use our zmax zinc coate coated zinccoate zinccoated connectors provide extra resistance against corrosion look z end model number versatile connector various 90 connections home repair projects stronger than angled nailing screw fastening alone help ensure joints consistently straight

In [133]:
idx_train = pd.read_pickle('LABELS_TRAIN.df')
idx_test = pd.read_pickle('LABELS_TEST.df')

label_train = idx_train['relevance']
idx_train.drop('relevance', axis=1, inplace=True)
idx_test.drop('relevance', axis=1, inplace=True)

In [136]:
df_train = idx_train.merge(queries, left_index=True, right_index=True)
df_test = idx_test.merge(queries, left_index=True, right_index=True)

In [ ]:
df_train = df_all.iloc[:num_train]
df_test = df_all.iloc[num_train:]
id_test = df_test['id']
y_train = df_train['relevance'].values
X_train = df_train[:]
X_test = df_test[:]

In [121]:
from sklearn.ensemble import RandomForestRegressor
#from sklearn import pipeline, model_selection
from sklearn import pipeline, grid_search
#from sklearn.feature_extraction import DictVectorizer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import TruncatedSVD
#from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error, make_scorer


def fmean_squared_error(ground_truth, predictions):
    fmean_squared_error_ = mean_squared_error(ground_truth, predictions)**0.5
    return fmean_squared_error_

def fmse(ground_truth, predictions):
    return mean_squared_error(ground_truth, predictions)

#RMSE  = make_scorer(fmse, greater_is_better=False)
RMSE  = make_scorer(fmean_squared_error, greater_is_better=False)

tfidf = TfidfVectorizer(ngram_range=(1, 2), stop_words='english')
tsvd = TruncatedSVD(n_components=10, random_state = 2016)
randomForestRegressor = RandomForestRegressor(n_estimators = 500, min_samples_leaf=3, n_jobs = -1, random_state = 5017, verbose = 1)

In [123]:
#query	product_uid	product_title	product_description	brand	attrs
class cust_regression_vals(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self
    def transform(self, hd_searches):
        d_col_drops=['query', 'product_uid', 'product_title','product_description', 'brand', 'attrs']
        hd_searches = hd_searches.drop(d_col_drops,axis=1).values
        return hd_searches

class cust_txt_col(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, x, y=None):
        return self
    def transform(self, data_dict):
        return data_dict[self.key].apply(str)

In [124]:
clf = pipeline.Pipeline([
        ('union', FeatureUnion(
                    transformer_list = [
                        ('cst',  cust_regression_vals()),  
                    
                        ('txt1', pipeline.Pipeline([('s1', cust_txt_col(key='query')), ('tfidf1', tfidf), ('tsvd1', tsvd)])),
                        ('txt2', pipeline.Pipeline([('s2', cust_txt_col(key='product_title')), ('tfidf2', tfidf), ('tsvd2', tsvd)])),
                        ('txt3', pipeline.Pipeline([('s3', cust_txt_col(key='product_description')), ('tfidf3', tfidf), ('tsvd3', tsvd)])),
                        ('txt4', pipeline.Pipeline([('s4', cust_txt_col(key='brand')), ('tfidf4', tfidf), ('tsvd4', tsvd)]))
                    
#                         ('txt1', pipeline.Pipeline([ ('s1', cust_txt_col(key='search_term')), ('tfidf1', tfidf)  ])),
#                         ('txt2', pipeline.Pipeline([ ('s2', cust_txt_col(key='product_title')), ('tfidf2', tfidf)  ])),
#                         ('txt3', pipeline.Pipeline([ ('s3', cust_txt_col(key='product_description')), ('tfidf3', tfidf) ])),
#                         ('txt4', pipeline.Pipeline([ ('s4', cust_txt_col(key='brand')), ('tfidf4', tfidf) ]))
                    
#                         ('brandf', pipeline.Pipeline([ ('s5', cust_txt_col(key='brand_feature')), ('ohenc', ohenc)  ])),
                        ],
                    transformer_weights = {
                        'cst': 1.0,
                        'txt1': 0.5,
                        'txt2': 0.25,
                        'txt3': 0.5,
                        'txt4': 0.5
#                         'brandf': 1.0
                        },
                n_jobs = -1
                ))
        , 
        ('rfr', randomForestRegressor)
    ])

#clf.set_params(rfr__max_features=10, rfr__max_depth=20)
#clf.fit(X_train, y_train)
# X_train

In [ ]:
start_time = time.time()

param_grid = {'rfr__max_features': [2], 'rfr__max_depth': [30]}
model = grid_search.GridSearchCV(estimator = clf, param_grid = param_grid, n_jobs = -1, cv = 5, verbose = 20, scoring=RMSE)
model.fit(X_train, y_train)

print("--- Training: %s minutes ---" % round(((time.time() - start_time)/60),2))

print("Best parameters found by grid search:")
print(model.best_params_)
print("Best CV score:")
print(model.best_score_)

In [137]:
a = clf.fit_transform(df_train)
#model.fit(X_train, y_train)
a

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          4.30054791e-04,  -2.73613362e-03,  -6.18544457e-03],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          4.30054524e-04,  -2.73613338e-03,  -6.18544478e-03],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -4.74867871e-03,  -7.39104102e-03,  -1.62204967e-03],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -2.95097968e-05,   7.62397356e-06,   1.99037905e-05],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          4.92271655e-15,   1.88133519e-15,   9.41396161e-16],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          7.33465945e-08,   5.12668454e-08,  -8.47681499e-08]])

In [146]:
np.any(np.isnan(label_train))

False

In [152]:
label_train

id
2         3.00
3         2.50
9         3.00
16        2.33
17        2.67
18        3.00
20        2.67
21        3.00
23        2.67
27        3.00
34        2.67
35        3.00
37        3.00
38        2.00
48        2.67
51        2.67
65        3.00
69        1.00
75        1.67
81        2.33
85        2.33
88        1.33
90        2.67
92        2.33
101       1.33
105       2.33
106       2.33
113       2.00
114       2.33
117       2.67
          ... 
221398    1.67
221400    1.33
221401    2.33
221404    2.33
221405    2.67
221407    1.67
221408    3.00
221409    2.67
221411    1.33
221412    2.00
221413    1.67
221415    2.00
221416    1.00
221419    2.33
221420    2.00
221422    2.33
221423    2.33
221426    2.00
221427    1.67
221432    2.00
221434    2.67
221443    3.00
221449    3.00
221450    2.00
221455    3.00
221457    1.00
221458    3.00
221463    2.33
221471    3.00
221473    2.33
Name: relevance, dtype: float64

In [151]:
label_train.iloc[[0,3,4,9]]

id
2     3.00
16    2.33
17    2.67
27    3.00
Name: relevance, dtype: float64

In [153]:
from sklearn.cross_validation import StratifiedKFold

skf = StratifiedKFold(label_train, n_folds=3, shuffle=True, random_state=117)
for train_index, test_index in skf:
    X_train, X_test = a[train_index], a[test_index]
    Y_train, Y_test = label_train.iloc[train_index], label_train.iloc[test_index]
    model = randomForestRegressor.fit(X_train, Y_train)
    yhat = model.predict(X_test)
    err = fmse(yhat, Y_test)
    print(err)

[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   42.2s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.0min finished
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   42.2s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.0min finished
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 500 out of 500 

0.223759904471
0.223653073192

[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   42.6s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.0min finished
[Parallel(n_jobs=16)]: Done  18 tasks      | elapsed:    0.0s
[Parallel(n_jobs=16)]: Done 168 tasks      | elapsed:    0.1s
[Parallel(n_jobs=16)]: Done 418 tasks      | elapsed:    0.3s
[Parallel(n_jobs=16)]: Done 500 out of 500 | elapsed:    0.3s finished



0.223125879838


In [154]:
np.save?

In [ ]:
np.save('FEATURES_1_TRAIN', a)
b = clf.fit_transform(df_test)
np.save('FEATURES_1_TEST', b)